# 02 · Modeling & Cross-Validation

## Objectives
- Establish grouped train/validation/test splits by cell.
- Benchmark baseline regressors and tree-based ensembles.
- Report metrics (RMSE, MAE, R²) with interpretation and error slices.

## Data Splitting Strategy
- TODO: Document grouped K-fold configuration (fold count, shuffle, seeds).
- TODO: Identify hold-out cell(s) reserved for final evaluation.
- TODO: Note any time-aware constraints to avoid leakage.

## Baseline Models
- TODO: Implement mean/median regressor baseline.
- TODO: Fit linear or elastic net model with core features.
- TODO: Capture baseline metrics for comparison.

## Tree-Based Models
- TODO: Train Random Forest Regressor (document hyperparameters).
- TODO: Train Gradient Boosting / HistGradientBoosting or XGBoost.
- TODO: Record cross-validation performance and variance.

## Hyperparameter Tuning
- TODO: Outline search space (RandomizedSearchCV/GridSearchCV).
- TODO: Capture best params and CV scores with mean ± std.
- TODO: Mention computational budget and seed control.

## Leaderboard
| Model | RMSE (CV) | MAE (CV) | R² (CV) | Notes |
| --- | --- | --- | --- | --- |
| Mean baseline |  |  |  |  |
| Elastic Net |  |  |  |  |
| Random Forest |  |  |  |  |
| Gradient Boosting |  |  |  |  |

## Hold-Out Evaluation
- TODO: Evaluate selected model(s) on hold-out cell(s).
- TODO: Report RMSE, MAE, R² with confidence intervals if available.
- TODO: Include parity plot placeholder (predicted vs. actual RUL).
- TODO: Document thresholded decision metrics (precision/recall) if applied.

## Interpretability & Diagnostics
- TODO: Compute permutation importance / feature importances.
- TODO: Add partial dependence or ALE plots for key features.
- TODO: Slice residuals by cell, C-rate, or RUL bands; log findings.

## Conclusions & Next Steps
- TODO: Summarize best-performing approach and limitations.
- TODO: List experiments to revisit or extend (different datasets, feature tweaks).